# Generating Word Embeddings - Lab

## Introduction

In this lab, you'll learn how to generate word embeddings by training a Word2Vec model, and then embedding layers into Deep Neural Networks for NLP!

## Objectives

You will be able to:

* Demonstrate a basic understanding of the architecture of the Word2Vec model
* Demonstrate an understanding of the various tunable parameters of word2vec such as vector size and window size

## Getting Started

In this lab, you'll start by creating your own word embeddings by making use of the Word2Vec Model. Then, you'll move onto building Neural Networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in your model. 

As you've seen, the easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which you can use to begin training immediately. For this lab, you'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.

Run the cell below to import everything you'll need for this lab. 

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize
import nltk

/opt/conda/envs/learn-env/lib/python3.6/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


Now, import the data. The data stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a github repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json` function from pandas to read the dataset into a DataFrame. Be sure to include the parameter `lines=True` when reading in the dataset!

Once you've loaded in the data, inspect the head of the DataFrame to see what your data looks like. 

In [2]:
!ls

CONTRIBUTING.md  LICENSE.md			News_Category_Dataset_v2.zip
index.ipynb	 News_Category_Dataset_v2.json	README.md


In [3]:
raw_df = pd.read_json("News_Category_Dataset_v2.json", lines = True)

In [4]:
raw_df.head()

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ..."


## Preparing the Data

Since you're working with text data, you need to do some basic preprocessing including tokenization. Notice from the data sample that two different columns contain text data--`headline` and `short_description`. The more text data your Word2Vec model has, the better it will perform. Therefore, you'll want to combine the two columns before tokenizing each comment and training your Word2Vec model. 

In the cell below:

* Create a column called `combined_text` that consists of the data from `df.headline` plus a space character (`' '`) plus the data from `df.short_description`.
* Use the `combined_text` column's `map()` function and pass in `word_tokenize`. Store the result returned in `data`.

In [5]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /home/gentle-
[nltk_data]     sailor-8400/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /home/gentle-
[nltk_data]     sailor-8400/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
from nltk.corpus import stopwords
import string
pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
#stop_words_list = stopwords.words("english")
stop_words_list = string.punctuation

def clean(list_): 
    list_ = nltk.regexp_tokenize(list_, pattern)
    list_ = [x.lower() for x in list_]
    list_ = list(filter(lambda w: w not in stop_words_list, list_))
    return list_
    
    

In [8]:
raw_df['combined_text'] = raw_df.headline + " " + raw_df.short_description
data = raw_df['combined_text'].map(word_tokenize)

Inspect the first 5 items in `data` to see how everything looks. 

In [9]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

Notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Remember that for a Word2Vec model you can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If your window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example you saw).

Now that you've prepared the data, train your model and explore a bit!

## Training the Model

Start by instantiating a Word2Vec Model from gensim below. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1`.
    * The number of threads to use during training, `workers=4`

In [10]:
model = Word2Vec(data, size = 100, window = 5, min_count = 1, workers = 4)

Now, that you've instantiated Word2Vec model, train it on your text data. 

In the cell below:

* Call `model.train()` and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which you can find in `model.corpus_count`. 
    * The number of `epochs` you want to train for, which we'll set to `10`

In [11]:
model.train(data,total_examples=model.corpus_count,epochs=10)

(55563930, 67352790)

Great! you now have a fully trained model! The word vectors themselves are stored inside of a `Word2VecKeyedVectors` instance, which is stored inside of `model.wv`. To simplify this, restore this object inside of the variable `wv` to save yourself some keystrokes down the line. 

In [12]:
wv = model.wv

## Examining Your Word Vectors

Now that you have a trained Word2Vec model, go ahead and explore the relationships between some of the words in the corpus! 

One cool thing you can use Word2Vec for is to get the most similar words to a given word. You can do this passing in the word to `wv.most_similar()`. 

In the cell below, try getting the most similar word to `'Texas'`.

In [14]:
wv.most_similar("Texas")

[('Maryland', 0.8117929100990295),
 ('Ohio', 0.8093268871307373),
 ('Georgia', 0.8071622848510742),
 ('Pennsylvania', 0.7993821501731873),
 ('Michigan', 0.7980077266693115),
 ('Arkansas', 0.7936064600944519),
 ('Louisiana', 0.7925847172737122),
 ('Arizona', 0.7893481254577637),
 ('Wisconsin', 0.787872314453125),
 ('Illinois', 0.7877964973449707)]

Interesting! All of the most similar words are also states. 

You can also get the least similar vectors to a given word by passing in the word to the `most_similar()` function's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [15]:
wv.most_similar(negative = "Texas")

[('Parent/Grandparent', 0.4148862659931183),
 ('uh-oh', 0.3941423296928406),
 ('Animal-rescue', 0.36208099126815796),
 ('much-vaunted', 0.3618099093437195),
 ('Sentiments', 0.35958802700042725),
 ('Equalize', 0.35745149850845337),
 ('maitre', 0.35658106207847595),
 ('Wars-Themed', 0.3560122549533844),
 ('PROVINCETOWN', 0.3520480990409851),
 ('Doze', 0.3493735194206238)]

This seems like random noise. It is a result of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with your given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

You can also get the vector for a given word by passing in the word as if you were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [16]:
wv["Texas"]

array([ 0.5145071 , -0.24826244, -0.03875696, -0.27601725,  0.6512589 ,
       -2.6371055 , -1.6626407 , -1.0076956 , -1.1000383 ,  0.0547162 ,
       -3.1519015 , -0.46951395, -2.4470649 ,  0.6309846 ,  2.3520262 ,
       -1.8777468 , -1.0648981 ,  1.4316293 ,  2.321335  , -0.14268252,
       -1.3431932 ,  0.9910789 , -0.80605465, -0.42213613,  0.3172755 ,
       -0.6324011 , -1.608554  , -0.05662008,  1.1680126 ,  0.21956988,
       -1.7765536 , -0.43441504,  0.5267735 ,  1.2780994 ,  0.6394775 ,
       -0.14637338,  0.8727525 ,  1.3105135 ,  1.9681015 ,  2.4116902 ,
       -1.0385612 , -0.9783772 , -1.8275063 ,  0.96550435,  1.9111084 ,
       -0.37606233,  0.36385316,  0.41224796, -0.605952  , -0.6571042 ,
       -1.2431523 ,  1.4820249 , -0.10939716,  0.07016461, -0.45288342,
        1.9266369 ,  0.5750974 ,  2.173643  ,  0.19608805,  0.166424  ,
        0.24426271, -1.1965159 , -0.7468506 ,  0.5546919 ,  4.078985  ,
        2.516191  ,  2.0593255 ,  0.64380085, -1.0175089 ,  1.36

Now get all of the word vectors from the object at once. You can find these inside of `wv.vectors`. Try it out in the cell below.  

In [17]:
wv.vectors

array([[-1.4342242e+00, -1.9059641e+00,  1.3206589e+00, ...,
        -4.3811893e-01, -8.7813395e-01, -1.2779821e+00],
       [-1.2761351e+00, -1.6338106e+00,  5.1533866e-01, ...,
        -1.2063392e+00, -1.4970827e+00, -9.5057827e-01],
       [-5.4786253e-01, -5.5249107e-01,  1.0767580e-01, ...,
        -7.4824160e-01, -1.1902704e+00, -5.6209022e-01],
       ...,
       [ 8.7477811e-02, -3.3519648e-02, -1.7606709e-02, ...,
         3.8684156e-02, -5.6506481e-02, -2.5268443e-02],
       [ 1.7352803e-03, -5.3722125e-02, -2.8414408e-02, ...,
         3.2642957e-02,  9.3985945e-03, -3.9057322e-02],
       [ 6.5271184e-02,  2.1567680e-03, -5.7724100e-02, ...,
         4.2700831e-02, -6.7252112e-03, -2.5046704e-02]], dtype=float32)

As a final exercise, try to recreate the _'king' - 'man' + 'woman' = 'queen'_ example previously mentioned. You can do this by using the `most_similar` function and translating the word analogies into an addition/subtraction formulation (as shown above). Pass the original comparison, which you are calculating a difference between, to the negative parameter, and the analogous starter you want to apply the same transformation to, to the `positive` parameter.

Do this now in the cell below. 

In [18]:
wv.most_similar(positive = ["king","woman"], negative = ["man"])

[('revival', 0.6386371850967407),
 ('princess', 0.6221472024917603),
 ('queen', 0.6210266947746277),
 ('brunette', 0.5906661748886108),
 ('supermodel', 0.5893117189407349),
 ('goddess', 0.5842194557189941),
 ('villain', 0.5783872008323669),
 ('title', 0.5745106339454651),
 ('epitome', 0.5685527324676514),
 ('diva', 0.5643763542175293)]

As you can see from the output above, your model isn't perfect, but 'Queen' is still in the top 3, and with 'Princess' not too far behind. As you can see from the word in first place, 'reminiscent', your model is far from perfect. This is likely because you didn't have enough training data. That said, given the small amount of training data provided, the model still performs remarkably well! 

In the next lab, you'll reinvestigate transfer learning, loading in the weights from an open-sourced model that has already been trained for a very long time on a massive amount of data. Specifically, you'll work with the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless your text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, you learned how to train and use a Word2Vec model to created vectorized word embeddings!